In [ ]:
#|default_exp test

In [ ]:
#|export
import time,os,sys,traceback,contextlib

from fastcore.utils import *
from fastcore.script import *
from fastcore.imports import *
from nbprocess.read import *
from nbprocess.doclinks import *
from nbprocess.process import NBProcessor
from warnings import warn

# Test Notebooks
> Run unit tests on notebooks in parallel

In [ ]:
#|export
def _do_eval(cell, flags):
    if cell.cell_type != 'code': return False
    direc = getattr(cell, 'directives_', {}) or {}
    if direc.get('eval:', [''])[0].lower() == 'false': return False
    return not flags & direc.keys()

def test_nb(fn, skip_flags=None, force_flags=None, do_print=False):
    "Execute tests in notebook in `fn` except those with `skip_flags`"
    if not IN_NOTEBOOK: os.environ["IN_TEST"] = '1'
    flags=set(L(skip_flags)) - set(L(force_flags))
    k,start = NBRunner(),time.time()
    
    def _exec_cell(cell):
        if _do_eval(cell, flags): k.run(cell)

    if do_print: print(f'Starting {fn}')
    try:
        with open(os.devnull, "w") as f, contextlib.redirect_stdout(f): NBProcessor(fn, _exec_cell).process()
        if do_print: print(f'- Completed {fn}')
        return True,time.time()-start
    except Exception as e:
        fence = '='*50
        msg = f'{e}\n{fence}' if str(e) else ''
        warn(f'\n\nError: {type(e).__name__} in {fn}:\n{fence}\n{msg}')
        traceback.print_exc()
        return False,time.time()-start

`test_nb` can test a notebook, and skip over certain flags:

In [ ]:
_nb = Path('../tests/directives.ipynb')
success,duration = test_nb(_nb, skip_flags=['notest'])
assert success
duration

0.0011682510375976562

Sometimes you may wish to override one or more of the skip_flags, in which case you can use the argument `force_flags` which will remove the appropriate tag(s) from `skip_flags`.  This is useful because `skip_flags` are meant to be set in the `tst_flags` field of `settings.ini`, whereas `force_flags` are usually passed in by the user.

In [ ]:
#|export
@call_parse
def nbprocess_test(
    fname:str=None,  # A notebook name or glob to convert
    flags:str='',  # Space separated list of test flags you want to run that are normally ignored
    n_workers:int=None,  # Number of workers to use
    timing:bool=False,  # Timing each notebook to see the ones are slow
    do_print:str=False, # Print start and end of each NB
    pause:float=0.01  # Pause time (in secs) between notebooks to avoid race conditions
):
    "Test in parallel the notebooks matching `fname`, passing along `flags`"
    skip_flags = config_key('tst_flags', '', path=False).split()
    force_flags = flags.split()
    files = [Path(f).absolute() for f in sorted(nbglob(fname))]
    if n_workers is None: n_workers = 0 if len(files)==1 else min(num_cpus(), 8)
    os.chdir(config_key("nbs_path"))
    results = parallel(test_nb, files, skip_flags=skip_flags, force_flags=force_flags, n_workers=n_workers, pause=pause, do_print=do_print)
    passed,times = zip(*results)
    if all(passed): print("Success.")
    else: 
        print("Failed:\n" + '\n'.join([f.name for p,f in zip(passed,files) if not p]))
        raise Exception('nbprocess tests failed.')
    if timing:
        for i,t in sorted(enumerate(times), key=lambda o:o[1], reverse=True): print(f"{files[i].name}: {int(t)} secs")

In [ ]:
#|eval:false
nbprocess_test(n_workers=0)

Success.


## Eval -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()